<a href="https://colab.research.google.com/github/IsraelAbebe/Personal-Projects-and-Exercises/blob/master/Paper-Play/Learn%2C_Imagine_and_Create_Text_to_Image_Generation_from_Prior_Knowledge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Learn, Imagine and Create: Text-to-Image
Generation from Prior Knowledge](http://papers.nips.cc/paper/8375-learn-imagine-and-create-text-to-image-generation-from-prior-knowledge.pdf)

In [0]:
%tensorflow_version 2.x
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from textblob import TextBlob


import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam,RMSprop,Adagrad
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
import PIL.Image as img
from PIL import Image


In [0]:
%%time
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, trainable=False, name='USE_embedding')
# Reduce logging output.
def get_sentese_embedding(messages):
    return embed(messages)

CPU times: user 13.4 s, sys: 1.94 s, total: 15.4 s
Wall time: 14.5 s


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
ls 'gdrive/My Drive/dataset/'

CACO/         GOOG.csv               sentiment/      Toxic/
dog.jpg       model.h5               submission.csv  train.csv
drugLib_raw/  sample_submission.csv  test.csv


In [0]:
dog_image = img.open('gdrive/My Drive/dataset/dog.jpg')

In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np

import torch.nn.functional as F

In [0]:
model = models.resnet50(pretrained=True)

In [0]:
# model

In [0]:
last_layer = nn.Linear(2048,512 ,bias=True)

model.fc = last_layer

In [0]:
def get_matching_score(sentence,image):
    sent_emb = get_sentese_embedding(sentence).numpy()

    transform = transforms.Compose([transforms.Resize(224),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor(),
                                             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
    img = Variable(transform(image))
    img = img.view(1, 3, 224, 224)
    img1_feature = model(img).detach().numpy()

    simmilarity_1 = tf.nn.softmax(np.dot(img1_feature.T,sent_emb))
    simmilarity_1 = simmilarity_1.numpy()

    visual_feature = np.dot(img1_feature,tf.nn.softmax(simmilarity_1))
    result = np.log(np.sum(np.exp(a*spatial.distance.cosine(visual_feature,sent_emb))))**(1/a)

    return result

In [0]:
get_matching_score(['the library is filled with mathimatics books. '],dog_image)

1.472069871421947e-10

In [0]:
get_matching_score(['this is a dog. it is furry and yellowish '],dog_image)

1.4588564277286308e-10

In [0]:
get_matching_score(['this is a dog. '],dog_image)


9.907701131327733e-11

In [0]:
get_matching_score(['dog.'],dog_image)

7.170906906119557e-11